# Feature Engineering pour la Détection de Fraudes

Ce notebook implémente les techniques de feature engineering pour améliorer la détection des fraudes.

## Objectifs

- Créer des features temporelles avancées
- Développer des indicateurs de comportement utilisateur
- Implémenter des scores de risque dynamiques
- Tester l'impact des nouvelles features sur les performances

In [ ]:
# Configuration et imports
import sys
import os
from pathlib import Path

# Ajout du chemin racine au sys.path
ROOT_DIR = Path.cwd().parent
sys.path.append(str(ROOT_DIR))

import warnings
warnings.filterwarnings('ignore')

# Imports de base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Imports ML
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Imports locaux
from src.data.data_loader import DataLoader
from src.data.preprocessor import Preprocessor
from src.data.feature_engineer import FeatureEngineer
from src.utils.metrics import calculate_metrics

# Configuration des graphiques
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Environnement configuré avec succès")

## 1. Chargement et Préparation des Données

In [ ]:
# Chargement des données
print("🔄 Chargement des données...")
data_loader = DataLoader()
df = data_loader.load_data(file_path="../data/raw/creditcard.csv")
print(f"📊 Données chargées : {df.shape}")
print(f"📈 Distribution des classes :\n{df['Class'].value_counts(normalize=True)}")

In [ ]:
# Aperçu des données
print("🔍 Aperçu des données :")
display(df.head())

print("\n📋 Informations sur les données :")
df.info()

print("\n📊 Statistiques descriptives :")
display(df.describe())

## 2. Feature Engineering de Base

In [ ]:
# Initialisation du Feature Engineer
feature_engineer = FeatureEngineer(save_path="../models/feature_engineer.pkl")

# Application du feature engineering
print("🔄 Application du feature engineering...")
df_features = feature_engineer.fit_transform(df.copy())
print(f"✅ Features créées : {df_features.shape[1]} colonnes")
print(f"📊 Nouvelles features ajoutées : {df_features.shape[1] - df.shape[1]}")

In [ ]:
# Affichage des nouvelles features
new_features = [col for col in df_features.columns if col not in df.columns]
print("🆕 Nouvelles features créées :")
for i, feature in enumerate(new_features, 1):
    print(f"{i:2d}. {feature}")

print(f"\n📊 Total : {len(new_features)} nouvelles features")

## 3. Analyse des Features Créées

In [ ]:
# Analyse de la distribution des nouvelles features
print("📊 Analyse des features temporelles :")

temporal_features = ['transaction_hour', 'transaction_day', 'transaction_month']
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, feature in enumerate(temporal_features):
    if feature in df_features.columns:
        df_features[feature].hist(bins=50, ax=axes[i], alpha=0.7)
        axes[i].set_title(f'Distribution de {feature}')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('Fréquence')

plt.tight_layout()
plt.show()

In [ ]:
# Analyse des features de fréquence
print("📊 Analyse des features de fréquence :")

frequency_features = ['frequency_last_1h', 'frequency_last_24h', 'transaction_velocity']
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, feature in enumerate(frequency_features):
    if feature in df_features.columns:
        # Filtrage des valeurs extrêmes pour une meilleure visualisation
        data_filtered = df_features[feature][df_features[feature] < df_features[feature].quantile(0.95)]
        data_filtered.hist(bins=30, ax=axes[i], alpha=0.7)
        axes[i].set_title(f'Distribution de {feature}')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('Fréquence')

plt.tight_layout()
plt.show()

In [ ]:
# Analyse des corrélations avec la cible
print("🔗 Analyse des corrélations avec la variable cible :")

# Sélection des features numériques
numeric_features = df_features.select_dtypes(include=[np.number]).columns
correlation_with_target = df_features[numeric_features].corr()['Class'].abs().sort_values(ascending=False)

print("Top 10 features les plus corrélées avec Class :")
display(correlation_with_target.head(10))

## 4. Feature Engineering Avancé

In [ ]:
# Création de features avancées
print("🔄 Création de features avancées...")

# Feature : Ratio Amount/Time
if 'Amount' in df_features.columns and 'Time' in df_features.columns:
    df_features['amount_per_second'] = df_features['Amount'] / (df_features['Time'] + 1)
    print("✅ Feature 'amount_per_second' créée")

# Feature : Score de risque composite
v_features = [col for col in df_features.columns if col.startswith('V')]
if len(v_features) > 5:
    df_features['risk_score_composite'] = df_features[v_features[:5]].mean(axis=1)
    print("✅ Feature 'risk_score_composite' créée")

# Feature : Anomalie temporelle
if 'transaction_hour' in df_features.columns:
    # Heures suspectes (nuit profonde)
    df_features['suspicious_hour'] = df_features['transaction_hour'].isin([1, 2, 3, 4, 5]).astype(int)
    print("✅ Feature 'suspicious_hour' créée")

print(f"📊 Total features après engineering avancé : {df_features.shape[1]}")

In [ ]:
# Analyse des nouvelles features avancées
advanced_features = ['amount_per_second', 'risk_score_composite', 'suspicious_hour']
existing_advanced = [f for f in advanced_features if f in df_features.columns]

if existing_advanced:
    print("📊 Analyse des features avancées :")
    
    fig, axes = plt.subplots(1, len(existing_advanced), figsize=(6*len(existing_advanced), 5))
    if len(existing_advanced) == 1:
        axes = [axes]
    
    for i, feature in enumerate(existing_advanced):
        if df_features[feature].dtype in ['int64', 'float64']:
            # Comparaison des distributions par classe
            sns.boxplot(data=df_features, x='Class', y=feature, ax=axes[i])
            axes[i].set_title(f'Distribution de {feature} par classe')
    
    plt.tight_layout()
    plt.show()

## 5. Sélection et Validation des Features

In [ ]:
# Préparation des données pour la validation
print("🔄 Préparation des données pour validation...")

# Séparation features/cible
X = df_features.drop('Class', axis=1)
y = df_features['Class']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📊 Dimensions d'entraînement : {X_train.shape}")
print(f"📊 Dimensions de test : {X_test.shape}")
print(f"📈 Distribution classes train : {y_train.value_counts(normalize=True).to_dict()}")
print(f"📈 Distribution classes test : {y_test.value_counts(normalize=True).to_dict()}")

In [ ]:
# Validation des features avec un modèle simple
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

print("🔄 Validation des features avec Random Forest...")

# Entraînement d'un modèle simple
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Prédictions
y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:, 1]

# Évaluation
print("📊 Rapport de classification :")
print(classification_report(y_test, y_pred))

print(".4f")
print(".4f")

# Features importantes
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\n🔝 Top 10 features importantes :")
display(feature_importance.head(10))

In [ ]:
# Visualisation des features importantes
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
sns.barplot(data=top_features, x='importance', y='feature')
plt.title('Top 15 Features Importantes')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

## 6. Sauvegarde et Export

In [ ]:
# Sauvegarde du dataset avec les nouvelles features
print("💾 Sauvegarde du dataset enrichi...")

output_path = "../data/processed/features_engineered.csv"
df_features.to_csv(output_path, index=False)
print(f"✅ Dataset sauvegardé : {output_path}")

# Sauvegarde des informations sur les features
features_info = {
    'original_features': list(df.columns),
    'engineered_features': new_features,
    'advanced_features': existing_advanced,
    'total_features': df_features.shape[1],
    'feature_importance': feature_importance.to_dict('records')
}

import json
with open("../models/features_info.json", 'w') as f:
    json.dump(features_info, f, indent=2, default=str)

print("✅ Informations sur les features sauvegardées")

## 7. Résumé et Conclusions

In [ ]:
print("🎯 RÉSUMÉ DU FEATURE ENGINEERING")
print("=" * 50)

print(f"📊 Données originales : {df.shape[1]} features")
print(f"📈 Données après engineering : {df_features.shape[1]} features")
print(f"🆕 Nouvelles features : {len(new_features)}")
print(f"🔧 Features avancées : {len(existing_advanced)}")

print("\n🏆 TOP 5 FEATURES LES PLUS IMPORTANTES :")
for i, (_, row) in enumerate(feature_importance.head(5).iterrows(), 1):
    print(".4f")

print("\n✅ OBJECTIFS ATTEINTS :")
print("• Features temporelles créées")
print("• Indicateurs de comportement implémentés")
print("• Scores de risque développés")
print("• Impact sur les performances validé")
print("• Pipeline de feature engineering opérationnel")

print("\n🚀 PRÊT POUR L'ENTRAÎNEMENT DES MODÈLES !")